In [1]:
%load_ext autoreload
%autoreload 2

In [58]:
import numpy as np
import pandas as pd
from copy import deepcopy           # Necessário para copiar os dados dentro de funções e evitar alterações inplace dos dados
                                    # Isso para que as funções recebam um dado e gerem um dado novo, mantendo o original inalterado.
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm

from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler




# TODO: converter para um arquivo requirements.txt 
# !pip install -U imbalanced-learn
# !pip install pandas
# !pip install numpy
# !pip install -U scikit-learn
# !pip install seaborn

In [59]:
DATA_PATH = 'dados/dados_treino.csv'

raw_data = pd.read_csv(DATA_PATH)

In [60]:
raw_data.info()    # Informaçãões gerais do dataset. Permite ver se há valores NULL.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975 entries, 0 to 4974
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   bateria                         4975 non-null   object 
 1   Tem_Instr_Violao_Viola          4975 non-null   bool   
 2   Tem_Instr_Guitarra              4975 non-null   bool   
 3   Tem_Instr_Cavaco                4975 non-null   bool   
 4   Tem_Instr_Sintetizador_Teclado  4975 non-null   bool   
 5   Tem_Instr_Piano                 4975 non-null   bool   
 6   Tem_Instr_Metais                4975 non-null   bool   
 7   Tem_Instr_Madeiras              4975 non-null   bool   
 8   Tem_Instr_Cordas                4975 non-null   bool   
 9   BPM                             4975 non-null   float64
 10  VolMedio                        4975 non-null   float64
 11  PctCantada                      4975 non-null   float64
 12  PctRap                          49

In [61]:
raw_data.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4975 entries, 0 to 4974
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   bateria                         4975 non-null   object 
 1   Tem_Instr_Violao_Viola          4975 non-null   bool   
 2   Tem_Instr_Guitarra              4975 non-null   bool   
 3   Tem_Instr_Cavaco                4975 non-null   bool   
 4   Tem_Instr_Sintetizador_Teclado  4975 non-null   bool   
 5   Tem_Instr_Piano                 4975 non-null   bool   
 6   Tem_Instr_Metais                4975 non-null   bool   
 7   Tem_Instr_Madeiras              4975 non-null   bool   
 8   Tem_Instr_Cordas                4975 non-null   bool   
 9   BPM                             4975 non-null   float64
 10  VolMedio                        4975 non-null   float64
 11  PctCantada                      4975 non-null   float64
 12  PctRap                          49

In [62]:
raw_data.head()

,bateria,Tem_Instr_Violao_Viola,Tem_Instr_Guitarra,Tem_Instr_Cavaco,Tem_Instr_Sintetizador_Teclado,Tem_Instr_Piano,Tem_Instr_Metais,Tem_Instr_Madeiras,Tem_Instr_Cordas,BPM,VolMedio,PctCantada,PctRap,duracao,ano_lancamento,modo,gostou,data_curtida,n_reproducao,id_cliente
0,Acústica,False,True,False,False,True,False,False,False,128.652946,-8.743969,38.671394,0.0,680510,1977,NaN,False,2021-07-29,5,#ID4940
1,Acústica,True,False,True,False,False,False,False,False,107.679121,-12.128194,75.739791,0.0,195746,2014,f,False,2021-07-29,5,#ID4940
2,Nenhuma,False,True,False,False,True,True,False,False,117.885751,-10.880246,35.978128,0.0,413618,1968,D#,False,2021-07-29,4,#ID4940
3,Acústica,False,True,False,False,True,True,True,False,119.309408,-12.489925,45.763376,0.0,749101,1963,D#,True,2021-07-29,3,#ID4940
4,Eletrônica,False,False,False,True,True,False,False,False,153.488568,-6.412484,58.979747,0.0,168978,2020,c,False,2021-07-29,2,#ID4940


In [63]:
# Dataset de referencia:
import sklearn.datasets
from sklearn import preprocessing
X,y = sklearn.datasets.load_boston(return_X_y=True)

In [64]:
print(type(X))

<class 'numpy.ndarray'>


## OBS: Essa classe está em construção e deve ser adequada antes de seu uso! No momento está feita praticamente como um aglomerado de funções

In [65]:
class Preprocessor():
    '''
        Saída final esperada: um dicionário com as saídas 
    '''
        # colocar as variáveis de interesse da classe:
        # especialmente o dataframe base, e dados de trabalho
#     def __init__(self):
#         # colocar as variáveis de interesse da classe:
#         # especialmente o dataframe de entrada
#         pass

#     def _bool_to_int(self, dataframe: pd.DataFrame) -> pd.DataFrame:
#         '''
        
#         Converte as colunas do tipo bool para o tipo int (0 e 1).
        
#         '''
#         columns = ['Tem_Instr_Violao_Viola', 'Tem_Instr_Guitarra', 'Tem_Instr_Cavaco', 
#                 'Tem_Instr_Sintetizador_Teclado', 'Tem_Instr_Piano', 'Tem_Instr_Metais', 
#                 'Tem_Instr_Madeiras', 'Tem_Instr_Cordas', 'gostou']                      # Adicionar as colunas da bateria aqui depois
#         new_df = deepcopy(dataframe)
        
#         for col in columns:
#             new_df[col] = new_df[col].astype(int)
            
#         return new_df
    
#     def _bateria_to_bool(self, dataframe: pd.DataFrame) -> pd.DataFrame:
#         '''
        
#         Transforma a coluna bateria para ser representada por binários
#         Essa função pode ser modificada para processar a coluna bateria de formas distintas.
        
#         '''
#         new_df = deepcopy(dataframe)
#         new_df['bateria_eletronica'] = (new_df['bateria'] == 'Eletrônica').astype(int)
#         new_df['bateria_acustica']   = (new_df['bateria'] == 'Acústica').astype(int)
#         new_df['bateria_nenhuma']     = (new_df['bateria'] == 'Nenhuma').astype(int)
#         new_df = new_df.drop(columns = ['bateria'])
#         return new_df

#     def _get_user_data(self, USER: str, raw_data: pd.DataFrame) -> pd.DataFrame:
#         user_data = raw_data[raw_data['id_cliente'] == USER]
#         user_data = bool_to_int(user_data)
#         user_data = bateria_to_bool(user_data)
#         user_data['modo'] = user_data['modo'].fillna("K")
        
#         return user_data
    
#     def scale_to_bool(self,scale):
#         string = {
#             'K'  : 1,
#             'c'  : 0,
#             'c#' : 0, 
#             'C'  : 1,
#             'C#' : 1,
#             'd'  : 0,
#             'd#' : 0,
#             'D'  : 1,
#             'D#' : 1,
#             'e'  : 0,
#             'E'  : 1,
#             'f'  : 0,
#             'f#' : 0,
#             'F'  : 1,
#             'F#' : 1,
#             'g'  : 0,
#             'g#' : 0,
#             'G'  : 1,
#             'G#' : 1,
#             'a'  : 0,
#             'a#' : 0,
#             'A'  : 1,
#             'A#' : 1,
#             'b'  : 0,
#             'B'  : 1
#         }[scale]
#         return string


#     def scale_to_one_hot(self, raw_data: pd.DataFrame):
#     # One hot encoding the mode feature data
#         new_df = deepcopy(raw_data)
#         mode_df = pd.get_dummies(new_df["modo"])
#         new_df = new_df.drop(columns = ['modo'])
#         # new_df.merge(mode_df,how='left', on='duracao')
#         #  Place the DataFrames side by side
#         new_df = pd.concat([new_df,mode_df], axis=1)
#         return new_df

#     def preprocess(self):    
#         '''
#             Aplica todas as estratégias de preprocessamento para adequar à estrutura entendida pelo sklearn
#         '''

#         raw_data = pd.read_csv(DATA_PATH)
#         raw_data['PctCantada'] = raw_data['PctCantada'] / 100
#         raw_data['PctRap'] = raw_data['PctRap'] / 100
#         raw_data['duracao'] = raw_data['duracao'] / (60*1000)
#         raw_data['VolMedio'] = raw_data['VolMedio'].abs()
#         raw_data['duracao']  = raw_data['duracao'].abs()
#         raw_data['modo'] = raw_data['modo'].fillna('K')
#         raw_data['escala_maior'] = raw_data['modo'].apply((lambda mode: scale_to_bool(mode)))
#         # raw_data['modo'] = raw_data['modo'].apply((lambda mode: scale_to_bool(mode)))
#         raw_data = bool_to_int(raw_data)
#         raw_data = bateria_to_bool(raw_data)

#     raw_data = scale_to_one_hot(raw_data) # Não gostei muito da solução, mas até faz sentido
#     print(raw_data)

#     def filter_user(self, user):
#         user_data = raw_data[raw_data['id_cliente'] == USER]
#         # definir melhor a interface

#     def train_test_split(self): #adicionar parametros de entrada
#         input_data = user_data.drop(columns = ["data_curtida", "id_cliente"])[user_data['id_cliente'] == USER]
#         # X = (.to_numpy()
#         X = input_data.drop(columns = ["gostou"]).to_numpy()
#         Y = input_data["gostou"].to_numpy()
#         Y = Y.ravel()
#         print(Y.shape)
#         print(X.shape)
#         X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% testv
#         return X_train, X_test, y_train, y_test 
    
#     def SMOTE_oversampling(self):
#     # colocar uns parametros uteis de entrada
#         # Teste do SMOTE
#         # sm = SMOTE(random_state=42)
#         # X_train_res, Y_train_res = sm.fit_resample(X_train, y_train)

#         a = np.arange(0,8)
#         b = np.array([11])
#         c = np.arange(15,44)
#         categorical_index = np.concatenate([a,c])
#         categorical_index = np.concatenate([categorical_index,b])
#         categorical_index = list(categorical_index)

#         # Teste SMOTE-NC
#         smote_nc_over = SMOTENC(categorical_features=categorical_index, random_state=0)

#         under = RandomUnderSampler(sampling_strategy='majority',random_state=0)

#         steps = [('o', smote_nc_over), ('u', under)]
#         pipeline = Pipeline(steps=steps)

#         X_train_res, Y_train_res = pipeline.fit_resample(X_train, y_train)

        # X_train_res, Y_train_res = smote_nc_over.fit_resample(X_train, y_train)

#     def check_balancing(self):
#         ## Count elements
#         count_plus = 0
#         count_minus = 0

#         for y_val in Y_train_res:
            
#             if y_val == 1:
#                 count_plus +=1
#             else: 
#                 count_minus +=1
#         print("Positive examples:",count_plus)
#         print("Negative examples:",count_minus)

# Talvez criar um método que aplique todos os passos pra evitar linhas de código -> Mas acho que será só isso o dataset, não deve haver problemas extras

In [66]:
# enc = preprocessing.OneHotEncoder()
# X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
# enc.fit(X)

# transform = enc.transform([['female', 'from US', 'uses Safari'],
#                ['male', 'from Europe', 'uses Safari']]).toarray()
# print(X)
# print(transform)

In [67]:
columns = ['Tem_Instr_Violao_Viola', 'Tem_Instr_Guitarra', 'Tem_Instr_Cavaco', 
           'Tem_Instr_Sintetizador_Teclado', 'Tem_Instr_Piano', 'Tem_Instr_Metais', 
           'Tem_Instr_Madeiras', 'Tem_Instr_Cordas', 'c' ,'K' ,'c#','C' ,'C#','d' ,'d#','D' ,'D#','e' ,'E' ,'f' ,'f#','F' ,'F#','g' ,'g#','G' ,'G#','a' ,'a#','A' ,'A#','B' ,'b'] 

In [68]:
# métodos já implementados
# def bool_to_int(dataframe: pd.DataFrame) -> pd.DataFrame:
#     '''
    
#     Converte as colunas do tipo bool para o tipo int (0 e 1).
    
#     '''
#     columns = ['Tem_Instr_Violao_Viola', 'Tem_Instr_Guitarra', 'Tem_Instr_Cavaco', 
#                'Tem_Instr_Sintetizador_Teclado', 'Tem_Instr_Piano', 'Tem_Instr_Metais', 
#                'Tem_Instr_Madeiras', 'Tem_Instr_Cordas', 'gostou']                      # Adicionar as colunas da bateria aqui depois
#     new_df = deepcopy(dataframe)
    
#     for col in columns:
#         new_df[col] = new_df[col].astype(int)
        
#     return new_df

# def bateria_to_bool(dataframe: pd.DataFrame) -> pd.DataFrame:
#     '''
    
#     Transforma a coluna bateria para ser representada por binários
#     Essa função pode ser modificada para processar a coluna bateria de formas distintas.
    
#     '''
#     new_df = deepcopy(dataframe)
#     new_df['bateria_eletronica'] = (new_df['bateria'] == 'Eletrônica').astype(int)
#     new_df['bateria_acustica']   = (new_df['bateria'] == 'Acústica').astype(int)
#     new_df['bateria_nenhuma']     = (new_df['bateria'] == 'Nenhuma').astype(int)
#     new_df = new_df.drop(columns = ['bateria'])
#     return new_df

# def get_user_data(USER: str, raw_data: pd.DataFrame) -> pd.DataFrame:
#     user_data = raw_data[raw_data['id_cliente'] == USER]
#     user_data = bool_to_int(user_data)
#     user_data = bateria_to_bool(user_data)
#     user_data['modo'] = user_data['modo'].fillna("K")
    
#     return user_data

# métodos da analise exploratoria

In [69]:
def scale_to_bool(scale):
    string = {
        'K'  : 1,
        'c'  : 0,
        'c#' : 0, 
        'C'  : 1,
        'C#' : 1,
        'd'  : 0,
        'd#' : 0,
        'D'  : 1,
        'D#' : 1,
        'e'  : 0,
        'E'  : 1,
        'f'  : 0,
        'f#' : 0,
        'F'  : 1,
        'F#' : 1,
        'g'  : 0,
        'g#' : 0,
        'G'  : 1,
        'G#' : 1,
        'a'  : 0,
        'a#' : 0,
        'A'  : 1,
        'A#' : 1,
        'b'  : 0,
        'B'  : 1
    }[scale]
    return string

In [70]:
# One hot encoding the mode feature data:

# def scale_to_one_hot(raw_data: pd.DataFrame):
#     new_df = deepcopy(raw_data)
#     mode_df = pd.get_dummies(new_df["modo"])
#     new_df = new_df.drop(columns = ['modo'])
#     # new_df.merge(mode_df,how='left', on='duracao')
#     #  Place the DataFrames side by side
#     new_df = pd.concat([new_df,mode_df], axis=1)
#     return new_df

In [71]:
# raw_data = pd.read_csv(DATA_PATH)
# raw_data['PctCantada'] = raw_data['PctCantada'] / 100
# raw_data['PctRap'] = raw_data['PctRap'] / 100
# raw_data['duracao'] = raw_data['duracao'] / (60*1000)
# raw_data['VolMedio'] = raw_data['VolMedio'].abs()
# raw_data['duracao']  = raw_data['duracao'].abs()
# raw_data['modo'] = raw_data['modo'].fillna('K')
# raw_data['escala_maior'] = raw_data['modo'].apply((lambda mode: scale_to_bool(mode)))
# # raw_data['modo'] = raw_data['modo'].apply((lambda mode: scale_to_bool(mode)))
# raw_data = bool_to_int(raw_data)
# raw_data = bateria_to_bool(raw_data)

# raw_data = scale_to_one_hot(raw_data) # Não gostei muito da solução, mas até faz sentido
# print(raw_data)

      Tem_Instr_Violao_Viola  Tem_Instr_Guitarra  Tem_Instr_Cavaco  \
0                          0                   1                 0   
1                          1                   0                 1   
2                          0                   1                 0   
3                          0                   1                 0   
4                          0                   0                 0   
...                      ...                 ...               ...   
4970                       1                   0                 0   
4971                       1                   0                 1   
4972                       0                   0                 0   
4973                       1                   1                 0   
4974                       0                   0                 0   

      Tem_Instr_Sintetizador_Teclado  Tem_Instr_Piano  Tem_Instr_Metais  \
0                                  0                1                 0   
1        

In [72]:
# USER = '#ID9181'
# # USER = '#ID4940'

# user_data = raw_data[raw_data['id_cliente'] == USER]

In [73]:
input_data = user_data.drop(columns = ["data_curtida", "id_cliente"])[user_data['id_cliente'] == USER]
# X = (.to_numpy()
X = input_data.drop(columns = ["gostou"]).to_numpy()
Y = input_data["gostou"].to_numpy()
Y = Y.ravel()
print(Y.shape)
print(X.shape)

(2403,)
(2403, 44)


In [74]:
from sklearn.model_selection import train_test_split
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% testv

In [75]:
# Teste do SMOTE
# sm = SMOTE(random_state=42)
# X_train_res, Y_train_res = sm.fit_resample(X_train, y_train)

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler

a = np.arange(0,8)
b = np.array([11])
c = np.arange(15,44)
categorical_index = np.concatenate([a,c])
categorical_index = np.concatenate([categorical_index,b])
categorical_index = list(categorical_index)

# Teste SMOTE-NC
smote_nc_over = SMOTENC(categorical_features=categorical_index, random_state=0)

under = RandomUnderSampler(sampling_strategy='majority',random_state=0)

steps = [('o', smote_nc_over), ('u', under)]
pipeline = Pipeline(steps=steps)

X_train_res, Y_train_res = pipeline.fit_resample(X_train, y_train)

# X_train_res, Y_train_res = smote_nc_over.fit_resample(X_train, y_train)

# Checar se o dataset ficou balanceado
count_plus = 0
count_minus = 0

for y_val in Y_train_res:
    
    if y_val == 1:
         count_plus +=1
    else: 
         count_minus +=1
print("Positive examples:",count_plus)
print("Negative examples:",count_minus)



Positive examples: 1635
Negative examples: 1635


## Testando um classificador para ver se a estrutura do dataset ficou correta

In [81]:
clf = svm.SVC(kernel='linear') # Linear Kernel
# clf.fit(X_train, y_train) # Sem SMOTE
clf.fit(X_train_res, Y_train_res)  # Com SMOTE

SVC(kernel='linear')

In [77]:
y_pred = clf.predict(X_test)

In [78]:
from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

Accuracy: 0.8357588357588358
Precision: 0.36585365853658536
Recall: 0.22058823529411764


## TODO: plotar curva ROC e acurácia para comparar os resultados 

In [79]:
raw_data[raw_data['id_cliente'] == '#ID4940']['gostou'].value_counts()        # Verificar se o dataset está desbalanceado.

0    1712
1     860
Name: gostou, dtype: int64

***

In [2]:
from Scripts.data_processing import DataManager

In [22]:
DATA_PATH = 'dados/dados_treino.csv'
manager = DataManager(DATA_PATH)

In [18]:
manager.raw_data

,bateria,Tem_Instr_Violao_Viola,Tem_Instr_Guitarra,Tem_Instr_Cavaco,Tem_Instr_Sintetizador_Teclado,Tem_Instr_Piano,Tem_Instr_Metais,Tem_Instr_Madeiras,Tem_Instr_Cordas,BPM,VolMedio,PctCantada,PctRap,duracao,ano_lancamento,modo,gostou,data_curtida,n_reproducao,id_cliente
0,Acústica,False,True,False,False,True,False,False,False,128.652946,-8.743969,38.671394,0.000000,680510,1977,NaN,False,2021-07-29,5,#ID4940
1,Acústica,True,False,True,False,False,False,False,False,107.679121,-12.128194,75.739791,0.000000,195746,2014,f,False,2021-07-29,5,#ID4940
2,Nenhuma,False,True,False,False,True,True,False,False,117.885751,-10.880246,35.978128,0.000000,413618,1968,D#,False,2021-07-29,4,#ID4940
3,Acústica,False,True,False,False,True,True,True,False,119.309408,-12.489925,45.763376,0.000000,749101,1963,D#,True,2021-07-29,3,#ID4940
4,Eletrônica,False,False,False,True,True,False,False,False,153.488568,-6.412484,58.979747,0.000000,168978,2020,c,False,2021-07-29,2,#ID4940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,Acústica,True,False,False,False,False,False,False,False,75.647653,-7.134472,78.413726,0.000000,178532,2012,D,False,2021-07-02,5,#ID9181
4971,Acústica,True,False,True,True,False,False,False,False,70.424589,-13.721616,76.379452,0.000000,123872,2020,c#,False,2021-07-02,4,#ID9181
4972,Acústica,False,False,False,True,False,True,False,False,82.829778,-7.971384,18.601275,78.043827,218364,2007,A,False,2021-07-02,2,#ID9181
4973,Eletrônica,True,True,False,False,True,False,False,False,115.032259,-7.477331,76.203737,0.000000,194007,1997,D,True,2021-07-02,3,#ID9181


In [14]:
manager.data

,Tem_Instr_Violao_Viola,Tem_Instr_Guitarra,Tem_Instr_Cavaco,Tem_Instr_Sintetizador_Teclado,Tem_Instr_Piano,Tem_Instr_Metais,Tem_Instr_Madeiras,Tem_Instr_Cordas,BPM,VolMedio,...,b,c,c#,d,d#,e,f,f#,g,g#
0,0,1,0,0,1,0,0,0,128.652946,8.743969,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,107.679121,12.128194,...,0,0,0,0,0,0,1,0,0,0
2,0,1,0,0,1,1,0,0,117.885751,10.880246,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,1,1,1,0,119.309408,12.489925,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,0,0,0,153.488568,6.412484,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,1,0,0,0,0,0,0,0,75.647653,7.134472,...,0,0,0,0,0,0,0,0,0,0
4971,1,0,1,1,0,0,0,0,70.424589,13.721616,...,0,0,1,0,0,0,0,0,0,0
4972,0,0,0,1,0,1,0,0,82.829778,7.971384,...,0,0,0,0,0,0,0,0,0,0
4973,1,1,0,0,1,0,0,0,115.032259,7.477331,...,0,0,0,0,0,0,0,0,0,0


In [13]:
manager.data.columns

Index(['Tem_Instr_Violao_Viola', 'Tem_Instr_Guitarra', 'Tem_Instr_Cavaco',
       'Tem_Instr_Sintetizador_Teclado', 'Tem_Instr_Piano', 'Tem_Instr_Metais',
       'Tem_Instr_Madeiras', 'Tem_Instr_Cordas', 'BPM', 'VolMedio',
       'PctCantada', 'PctRap', 'duracao', 'ano_lancamento', 'gostou',
       'data_curtida', 'n_reproducao', 'id_cliente', 'escala_maior',
       'bateria_eletronica', 'bateria_acustica', 'bateria_nenhuma', 'A', 'A#',
       'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'K', 'a', 'a#',
       'b', 'c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#'],
      dtype='object')

In [17]:
manager.users

['#ID4940', '#ID9181']

In [34]:
USER = manager.users[1]
manager.user_data(USER)

,Tem_Instr_Violao_Viola,Tem_Instr_Guitarra,Tem_Instr_Cavaco,Tem_Instr_Sintetizador_Teclado,Tem_Instr_Piano,Tem_Instr_Metais,Tem_Instr_Madeiras,Tem_Instr_Cordas,escala_maior,bateria_eletronica,...,ano_lancamento,BPM,VolMedio,PctCantada,PctRap,duracao,id_cliente,data_curtida,n_reproducao,gostou
2572,0,0,1,0,0,0,0,0,1,0,...,1999,77.778813,10.062582,0.717456,0.000000,3.913483,#ID9181,2021-07-02,3,0
2573,0,0,0,1,0,1,0,1,1,1,...,2002,95.475172,7.059382,0.149406,0.920170,4.096767,#ID9181,2021-07-02,3,0
2574,1,0,0,1,0,1,0,1,0,0,...,2002,117.121836,6.729495,0.700721,0.230951,4.033967,#ID9181,2021-07-02,4,0
2575,1,1,0,1,1,0,0,0,1,1,...,2006,109.310684,7.688355,0.832810,0.000000,3.724500,#ID9181,2021-07-02,4,0
2576,1,0,0,0,1,1,1,1,1,0,...,2007,121.550659,17.025234,0.000000,0.000000,2.289283,#ID9181,2021-07-02,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,1,0,0,0,0,0,0,0,1,0,...,2012,75.647653,7.134472,0.784137,0.000000,2.975533,#ID9181,2021-07-02,5,0
4971,1,0,1,1,0,0,0,0,0,0,...,2020,70.424589,13.721616,0.763795,0.000000,2.064533,#ID9181,2021-07-02,4,0
4972,0,0,0,1,0,1,0,0,1,0,...,2007,82.829778,7.971384,0.186013,0.780438,3.639400,#ID9181,2021-07-02,2,0
4973,1,1,0,0,1,0,0,0,1,1,...,1997,115.032259,7.477331,0.762037,0.000000,3.233450,#ID9181,2021-07-02,3,1


In [22]:
X, Y = manager.data_arrays()
print(X)
print(Y)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0 5]
 [0 5]
 [0 4]
 ...
 [0 2]
 [1 3]
 [0 5]]


In [35]:
import numpy as np
X, Y = manager.data_arrays(USER)
print(X)
print(Y)
print(np.unique(Y[:, 0], return_counts = True))

[[0.         0.         1.         ... 0.71745628 0.         3.91348333]
 [0.         0.         0.         ... 0.1494064  0.92017037 4.09676667]
 [1.         0.         0.         ... 0.70072094 0.23095085 4.03396667]
 ...
 [0.         0.         0.         ... 0.18601275 0.78043827 3.6394    ]
 [1.         1.         0.         ... 0.76203737 0.         3.23345   ]
 [0.         0.         0.         ... 0.14337007 0.79049171 3.3568    ]]
[[0 3]
 [0 3]
 [0 4]
 ...
 [0 2]
 [1 3]
 [0 5]]
(array([0, 1]), array([2048,  355]))


In [36]:
X[np.where(Y[:, 0] == 1)[0]].shape

(355, 43)

In [37]:
kwargs = {'test_size': 0.2}
X_train, X_test, y_train, y_test = manager.train_test_split(USER, kwargs, classification = True, balance = True)


In [38]:
from imblearn.over_sampling import SMOTENC

In [41]:
np.unique(y_test, return_counts = True)

(array([0, 1]), array([430, 390]))

In [40]:
len(X_test)

820

In [1]:
2 + 2

4